In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [2]:
import logging
import pandas as pd
import delta_sharing

import general_functions.databricks_client as db_client
from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url
from general_functions.call_api_with_account_id import send_to_innkeepr_api_paginated

In [3]:
customer = "Autobatterienbilliger" #6617da9c01c2ab3bd12c63cb
date = "2026-02-01"#T20:00:00.000Z"
end_date = "2026-02-02"#T27T00:00:00.000Z"
use_session_id = False
url = return_api_url()
print(f"url = {url}")
account_id = return_account_ids()
account_id = [acc["id"] for acc in account_id if acc["name"] == customer]
account_id = account_id[0]

In [4]:
if use_session_id:
    profile_path = db_client.return_databricks_client()
    table_path = f"{profile_path}#delta_share_events.{account_id}.features_view_30_outlook"
    df = delta_sharing.load_as_pandas(table_path, limit=500000)
    df = df[df["created"].astype("string")>=date]
    print(df["created"].min(), df["created"].max())
    session_ids=df["session"].unique().tolist()
    print(f"len session_ids = {len(session_ids)}")
    content = {"sessionId":session_ids}
else:
    content = {"created":{"$gte":date, "$lt":end_date}}

In [5]:
sessions_response = send_to_innkeepr_api_paginated( 
    f"{url}/sessions/query",
    account_id,
    content,
    logging
)

In [6]:
sessions = pd.json_normalize(sessions_response)
sessions

In [11]:
sessions["campaign.inkpr_campaign_id"].value_counts()

In [ ]:
sessions.columns

In [ ]:
sessions.to_parquet(f"DataChecks/sessions/test_sessions_{customer}.parquet")

In [10]:
### customer specific
invalid_ids = sessions[sessions["campaign.inkpr_campaign_id"]=="{campaign.id}"]["campaign.gclid"]
signals_filtered = signals[signals["externalId"].isin(invalid_ids)]
signals_filtered["relates_to.campaign.name"].value_counts()

In [ ]:
signals_filtered

# Some Checks

## check signals

In [7]:
signals = send_to_innkeepr_api_paginated(
    f"{url}/signals/query",
    account_id,
    #{"externalId":["1435494216", "267300181"]},
    {"externalId": sessions["campaign.gclid"].dropna().unique().tolist()},
    #{"adAccountId":"552565349"},
    logging
)

signals = pd.json_normalize(signals)
signals

In [ ]:
signals["source"].value_counts()

In [ ]:
treamtents = send_to_innkeepr_api_paginated(
    f"{url}/treatments/query",
    account_id,
    #{"externalId": sessions["campaign.utm_id"].dropna().unique().tolist()},
    #{"externalId":["1435494216", "267300181"]},
    {},
    logging
)

treamtents = pd.json_normalize(treamtents)
treamtents

In [ ]:
for col in treamtents:
    for signal in ["1435494216", "267300181"]:
        temp = treamtents[treamtents[col].astype("str").str.contains(signal)]
        if len(temp)>0:
            print(col, signal, len(temp))

## check with conversions

In [ ]:
from general_functions.datetime_helper import transform_date_to_timestamp_milliseconds

test = sessions[sessions["campaign.ad_id"].isnull()==False]
meta_sessions = test["sessionId"].dropna().unique().tolist()
content = {
     "created": {
                     "$gte": transform_date_to_timestamp_milliseconds("20250818"),
                     "$lte": transform_date_to_timestamp_milliseconds("20250819"),
               }
    }
conversions = send_to_innkeepr_api_paginated(
    f"{url}/conversions/query",
    account_id,
    content,
    logging
    
)

conversions = pd.json_normalize(conversions)
conversions

In [ ]:
conversions_matches = conversions[conversions["sessionId"].isin(meta_sessions)]
conversions_matches

In [ ]:
externalIds = sessions[sessions["sessionId"].isin(conversions_matches["sessionId"])]["campaign.utm_id"].dropna().unique().tolist()
len(externalIds), externalIds

In [ ]:
signals = send_to_innkeepr_api_paginated(
    f"{url}/signals/query",
    account_id,
    {"externalId":externalIds},
    logging
)

signals = pd.json_normalize(signals)
signals

In [ ]:
treatments = send_to_innkeepr_api_paginated(
    f"{url}/treatments/query",
    account_id,
    {"id":signals["relates_to.treatment"].dropna().unique().tolist()},
    logging
)
treatments = pd.json_normalize(treatments)
treatments

# chech sessions and gclids

In [ ]:
sessions_with_conversions = pd.merge(sessions, conversions_matches, how="left", left_on="sessionId", right_on="sessionId")
sessions_with_gclid = sessions_with_conversions[sessions_with_conversions["campaign.gclid"].isnull() == False]